# Importação

In [22]:
import pandas as pd
import pyomo.environ as pyo 
from pyomo.environ import SolverFactory
from pyomo.environ import Var, Integers, Binary, Constraint,TransformationFactory,RangeSet
from datetime import time, datetime
#import highspy
#import gurobipy

# Timestamp da rodada
P_HoraDaRodada = datetime.now()
P_HoraDaRodadaTimeStamp = int(P_HoraDaRodada.strftime("%Y%m%d%H%M%S"))

# Inicialização do modelo
model = pyo.ConcreteModel()
solver_selecionado = 'cbc'
relaxacao_linear = 0 # 0 = MIP, 1 = Relaxação linear

# importação de libs específicas de acordo com o solver
#if solver_selecionado == 'highs':
    #import highspy
#if solver_selecionado == 'gurobi':
    #import gurobipy
if solver_selecionado == 'mosek':
    import mosek

# Inicialização de dicionários
Ps_NomeDaRodada = {}
P_SOL_incumbent_solution = {}
P_SOL_solver_status = {}
P_SOL_solver_time = {}
P_SOL_best_bound = {}
P_SOL_optimality_gap = {}
start_time = {}
end_time = {}
elapsed_time = {}
P_total_variables = {}
P_integer_variables = {}
P_binary_variables = {}
P_total_constraints = {}

# Funções de apoio
def Func_ConversaoDatetimeParaMinutosEscalares(P_ARG_Datetime):
    P_Minutos = P_ARG_Datetime.hour * 60 + P_ARG_Datetime.minute
    return P_Minutos

def Func_ConversaoDatetimeParaMinutosParametros(data_dict):
    return {
        key: (val.hour * 60 + val.minute if isinstance(val, time) else val)
        for key, val in data_dict.items()
    }

# Recursão

In [23]:
S_Instancias = ["i09","i10","i10C","i11","i12","i13","i14C","i15","i25","i30","i37","i38","i45","i46C","i50"]
S_InstanciasMuitoPequenas = ["i09","i10","i10C","i11"]
S_InstanciasPequenas = ["i09","i10","i10C","i11","i12","i13"]
S_InstanciasGrandes = ["i25","i30","i37","i38","i45","i46C","i50"]
S_InstanciasRodada = ["i25","i30","i37","i38"]
S_Teste =  ["i09"]

## Início da recursão #####################################################################################################################
for instancia in S_Teste:
    
    # IMPORTAÇÃO ###############################################################################################################################3
    
    #1 Leitura do arquivo .xslx para conjuntos e parâmetros
    Ps_CaminhoDaPlanilha = f"G:\Meu Drive\Planilhas para Estudos\{instancia}.xlsx"
    #1.1 Escalares
    df_escalares = pd.read_excel(Ps_CaminhoDaPlanilha, sheet_name = "Escalares")
    #print(df_escalares)
    #print(df_escalares.dtypes)

    #1.2 Programacao Inicial
    df_proginicial = pd.read_excel(Ps_CaminhoDaPlanilha, sheet_name = "ProgramacaoInicial")
    #print(df_proginicial)
    #print(df_proginicial.dtypes)

    #1.3 Aeronaves
    df_aeronaves = pd.read_excel(Ps_CaminhoDaPlanilha, sheet_name = "Aeronaves")
    #print(df_aeronaves)
    #print(df_aeronaves.dtypes)

    #1.1 Compatibilidade
    df_compatibilidade = pd.read_excel(Ps_CaminhoDaPlanilha, sheet_name = "Compatibilidade")
    #print(df_compatibilidade)
    #print(df_compatibilidade.dtypes)

    ########################################################## Conjuntos
    # 2 Criação de conjuntos e parametros
    # 2.1 Conjuntos

    #Voos
    # Todos
    model.S_VoosReais = pyo.Set(initialize=df_proginicial.Voo)
    S_VoosReais = model.S_VoosReais
    #print(list(S_VoosReais))
    # Comitiva
    model.S_VoosComitiva = pyo.Set(
        within=model.S_VoosReais, 
        initialize=df_proginicial[df_proginicial['TipoDoVoo'] == "Tab.Comitivas"]['Voo'].tolist()
    )
    S_VoosComitiva = model.S_VoosComitiva
    # Transferência 2
    model.S_VoosT2 = pyo.Set(
        within=model.S_VoosReais, 
        initialize=df_proginicial[df_proginicial['TipoDoVoo'] == "Transf.2"]['Voo'].tolist()
    )
    S_VoosT2 = model.S_VoosT2
    # Transferência 1
    model.S_VoosT1 = pyo.Set(
        within=model.S_VoosReais, 
        initialize=df_proginicial[df_proginicial['TipoDoVoo'] == "Transf.1"]['Voo'].tolist()
    )
    S_VoosT1 = model.S_VoosT1
    # Transferência todos
    S_VoosTransferencia = S_VoosT1.union(S_VoosT2)
    # Tabela
    model.S_VoosTabela = pyo.Set(
        within=model.S_VoosReais, 
        initialize=df_proginicial[df_proginicial['TipoDoVoo'] == "Tab."]['Voo'].tolist()
    )
    S_VoosTabela = model.S_VoosTabela
    # Tabela + Comitiva
    S_VoosTabelaComitivaLista = S_VoosTabela.union(S_VoosComitiva)
    model.S_VoosTabelaComitiva = pyo.Set(
        initialize=S_VoosTabelaComitivaLista
    )
    S_VoosTabelaComitiva = model.S_VoosTabelaComitiva
    print(list(S_VoosTabelaComitiva))

    # Outros subconjuntos de voos usados no modelo
    # Reais exceto voos comitiva
    S_VoosReaisSemComitiva= S_VoosReais - S_VoosComitiva
    #print("S_VoosReaisSemComitiva=", list(S_VoosReaisSemComitiva))

    #Unidades Marítimas
    model.S_UM = pyo.Set(initialize=df_proginicial.UMDestino)
    S_UM = model.S_UM

    #Aeronaves
    #Geral
    model.S_Aeronaves = pyo.Set(initialize=df_aeronaves.Aeronave)
    S_Aeronaves = model.S_Aeronaves 
    # Spot
    model.S_AeronavesSpot= pyo.Set(
        within=model.S_Aeronaves, 
        initialize=df_aeronaves[df_aeronaves['TipoDaAeronave'] == 3]['Aeronave'].tolist()
    )
    S_AeronavesSpot = model.S_AeronavesSpot
    # Pool
    model.S_AeronavesPool= pyo.Set(
        within=model.S_Aeronaves, 
        initialize=df_aeronaves[df_aeronaves['TipoDaAeronave'] == 2]['Aeronave'].tolist()
    )
    S_AeronavesPool = model.S_AeronavesPool
    # Comum
    model.S_AeronavesComum= pyo.Set(
        within=model.S_Aeronaves, 
        initialize=df_aeronaves[df_aeronaves['TipoDaAeronave'] == 1]['Aeronave'].tolist()
    )
    S_AeronavesComum = model.S_AeronavesComum

    ##### Parâmetros
    #2.2.1 Escalares usando param
    Ps_NomeDaInstancia = df_escalares.NomeDaInstancia[0]  
    print("Ps_NomeDaInstancia = ",Ps_NomeDaInstancia)

    P_NumeroDeVoos= df_escalares.NumeroDeVoos[0]
    #print("P_NumeroDeVoos = ",P_NumeroDeVoos)

    P_TempoDeSeguranca = df_escalares.TempoDeSeguranca[0]
    #print("P_TempoDeSeguranca = ",P_TempoDeSeguranca)

    P_NumeroMaxDeVoosPorAeronave = df_escalares.NumeroMaxDeVoosPorAeronave[0]
    #print("P_NumeroMaxDeVoosPorAeronave = ",P_NumeroMaxDeVoosPorAeronave)
    
    ######################################################################
    # Importação do conjunto de Posições, que é dependente do parâmetro a seguir
    S_PosicoesLista = list(range(1, P_NumeroMaxDeVoosPorAeronave+1))
    #print("S_PosicoesLista = ", S_PosicoesLista)

    model.S_Posicoes = pyo.Set(initialize=S_PosicoesLista)
    S_Posicoes = model.S_Posicoes
    print("S_Posicoes = ", list(S_Posicoes))
    model.S_PosicoesExcetoUltima = pyo.Set(initialize=S_PosicoesLista[:-1])
    S_PosicoesExcetoUltima = model.S_PosicoesExcetoUltima
    print("S_PosicoesExcetoUltima = ", list(S_PosicoesExcetoUltima))
    ######################################################################

    P_TurnaroundTime = df_escalares.TurnaroundTime[0]
    #print("P_TurnaroundTime = ",P_TurnaroundTime)

    P_AtrasoLimite1 = df_escalares.AtrasoLimite1[0]  
    #print("P_AtrasoLimite1 = ",P_AtrasoLimite1)

    P_AtrasoLimite2 = df_escalares.AtrasoLimite2[0]  
    #print("P_AtrasoLimite2 = ",P_AtrasoLimite2)

    P_InicioDoHorizonteDatetime = df_escalares.InicioDoHorizonte[0]
    #print("P_InicioDoHorizonteDatetime = ",P_InicioDoHorizonteDatetime)
    P_InicioDoHorizonte = Func_ConversaoDatetimeParaMinutosEscalares(P_InicioDoHorizonteDatetime)
    #print("P_InicioDoHorizonte = ",P_InicioDoHorizonte)

    P_FimDoHorizonteDatetime = df_escalares.FimDoHorizonte[0]
    #print("P_FimDoHorizonteDatetime = ",P_FimDoHorizonteDatetime)
    P_FimDoHorizonte = Func_ConversaoDatetimeParaMinutosEscalares(P_FimDoHorizonteDatetime)
    #print("P_FimDoHorizonte = ",P_FimDoHorizonte)

    P_PenCancelamentoComitiva = df_escalares.PenCancelamentoComitiva[0] 
    #print("P_PenCancelamentoComitiva = ",P_PenCancelamentoComitiva.value)

    P_PenCancelamento2Dias = df_escalares.PenCancelamento2Dias[0]
    #print("P_PenCancelamento2Dias = ",P_PenCancelamento2Dias.value)

    P_PenCancelamento1Dia = df_escalares.PenCancelamento1Dia[0]
    #print("P_PenCancelamento1Dia = ",P_PenCancelamento1Dia.value)

    P_PenCancelamentoTabela = df_escalares.PenCancelamentoTabela[0] 
    #print("P_PenCancelamentoTabela = ",P_PenCancelamentoTabela.value)

    P_PenUsoAeronaveSpot = df_escalares.PenUsoAeronaveSpot[0]
    #print("P_PenUsoAeronaveSpot = ",P_PenUsoAeronaveSpot.value)

    P_PenUsoAeronavePool = df_escalares.PenUsoAeronavePool[0]
    #print("P_PenUsoAeronavePool = ",P_PenUsoAeronavePool.value)

    P_PenUsoAeronaveComum = df_escalares.PenUsoAeronaveComum[0] 
    #print("P_PenUsoAeronaveComum = ",P_PenUsoAeronaveComum.value)

    P_PenTrocaAeronave = df_escalares.PenTrocaAeronave[0] 
    #print("P_PenTrocaAeronave = ",P_PenTrocaAeronave.value)

    P_PenAtrasoTipo2 = df_escalares.PenAtrasoTipo2[0]
    #print("P_PenAtrasoTipo2 = ",P_PenAtrasoTipo2.value)

    P_PenAtrasoTipo1 = df_escalares.PenAtrasoTipo1[0]
    #print("P_PenAtrasoTipo1 = ",P_PenAtrasoTipo1.value)

    P_PenAtrasoMinuto = df_escalares.PenAtrasoMinuto[0]
    #print("P_PenAtrasoMinuto = ",P_PenAtrasoMinuto.value)

    #2.2.2 Indexados
    #Convertendo para um dicionário, mas o índice deve ser igual ao nome da coluna
    #Horário Inicial
    #   Conversao de datetime para minutos
    P_HorarioInicialDatetime_dict = df_proginicial.set_index('Voo')['HorarioInicial'].to_dict()
    P_HorarioInicial_dict = Func_ConversaoDatetimeParaMinutosParametros(P_HorarioInicialDatetime_dict)
    #   Alteração do horário inicial para ser sempre 07:00
    for voo in S_VoosReais:
        if P_HorarioInicial_dict[voo] < 420:
            P_HorarioInicial_dict[voo] = 420

    #   Transformação de lista em parâmetro para modelo
    model.P_HorarioInicial = pyo.Param(S_VoosReais, initialize=P_HorarioInicial_dict)
    P_HorarioInicial = model.P_HorarioInicial
    #print("P_HorarioInicial = ",P_HorarioInicial.display())

    #UM Destino
    P_UMDestino_dict = df_proginicial.set_index('Voo')['UMDestino'].to_dict()
    model.P_UMDestino = pyo.Param(S_VoosReais, initialize=P_UMDestino_dict)
    P_UMDestino = model.P_UMDestino  
    #print("P_UMDestino = ",P_UMDestino.display())

    #Duração do Voo
    #   Conversao de datetime para minutos
    P_DuracaoDoVooDatetime_dict = df_proginicial.set_index('Voo')['DuracaoDoVoo'].to_dict()
    P_DuracaoDoVoo_dict = Func_ConversaoDatetimeParaMinutosParametros(P_DuracaoDoVooDatetime_dict)
    #   Transformação de lista em parâmetro para modelo
    model.P_DuracaoDoVoo = pyo.Param(S_VoosReais, initialize=P_DuracaoDoVoo_dict)
    P_DuracaoDoVoo = model.P_DuracaoDoVoo  
    print("P_DuracaoDoVoo= ",P_DuracaoDoVoo.display())

    # Aeronave inicial
    P_AeronaveInicialString_dict = df_proginicial.set_index('Voo')['AeronaveInicial'].to_dict()
    # Convertendo para um parâmetro de 2 índices binário, em formato de dicionário
    P_AeronaveInicial = {}
    for i in S_VoosReais:
        for h in S_Aeronaves:
            if P_AeronaveInicialString_dict[i] == h:
                P_AeronaveInicial[i,h] = 1
            else:
                P_AeronaveInicial[i,h] = 0

    # Declarando o dicionário como parâmetro         
    model.P_AeronaveInicial = pyo.Param(S_VoosReais, S_Aeronaves, initialize=P_AeronaveInicial)
    P_AeronaveInicial = model.P_AeronaveInicial
    #print("P_AeronaveInicial = ",P_AeronaveInicial.display())

    # Tempo de Permanência
    #   Conversao de datetime para minutos
    P_TempoDePermanenciaDatetime_dict = df_proginicial.set_index('Voo')['TempoDePermanencia'].to_dict()
    P_TempoDePermanencia_dict = Func_ConversaoDatetimeParaMinutosParametros(P_TempoDePermanenciaDatetime_dict)
    #   Transformação de lista em parâmetro para modelo
    model.P_TempoDePermanencia = pyo.Param(S_VoosReais, initialize=P_TempoDePermanencia_dict)
    P_TempoDePermanencia = model.P_TempoDePermanencia 
    #print("P_TempoDePermanencia = ",P_TempoDePermanencia.display())

    # Tipo de Aeronave
    P_TipoDeAeronave_dict = df_aeronaves.set_index('Aeronave')['TipoDaAeronave'].to_dict()
    model.P_TipoDeAeronave = pyo.Param(S_Aeronaves, initialize=P_TipoDeAeronave_dict)
    P_TipoDeAeronave = model.P_TipoDeAeronave 
    #print("P_TipoDeAeronave = ",P_TipoDeAeronave.display())

    # Compatibilidade
    # Como esse parâmetro possui mais de um índice, é necessária uma manipulação prévia
    # Além disso, o dataframe com os dados deste parâmetro está num formato em que um dos indices (S_Voos) está na coluna e o outro (S_Aeronaves) está como cabeçalho
    # Para isso, os passos abaixo são seguidos:
    # Passo 1) usar a função .melt para alterar o formato do dataframe, transferindo o cabeçalho para uma nova coluna:
    df_compatibilidade_mod= df_compatibilidade.melt(id_vars=['Voo'], var_name='Aeronaves', value_name='Compatibilidade')
    # Passo 2) Create the dictionary in the (S_VoosReais, S_UM): Value format, que é o formato que a função pyo.Param aceita os dados de um parâmetro multiíndice
    P_Compatibilidade_dict = {(row['Voo'], row['Aeronaves']): row['Compatibilidade'] for _, row in df_compatibilidade_mod.iterrows()}
    # Passo 3) Por fim, realizar a atribuição ao parâmetro normalmente
    model.P_Compatibilidade = pyo.Param(S_VoosReais, S_Aeronaves, initialize=P_Compatibilidade_dict)
    P_Compatibilidade = model.P_Compatibilidade 
    # print("P_Compatibilidade = ",P_Compatibilidade.display())

    #2.2.1 Escalares usando param
    Ps_NomeDaInstancia = df_escalares.NomeDaInstancia[0]  
    print("Ps_NomeDaInstancia = ",Ps_NomeDaInstancia)

    P_NumeroDeVoos= df_escalares.NumeroDeVoos[0]
    #print("P_NumeroDeVoos = ",P_NumeroDeVoos)

    P_TempoDeSeguranca = df_escalares.TempoDeSeguranca[0]
    #print("P_TempoDeSeguranca = ",P_TempoDeSeguranca)

    P_NumeroMaxDeVoosPorAeronave = df_escalares.NumeroMaxDeVoosPorAeronave[0]
    #print("P_NumeroMaxDeVoosPorAeronave = ",P_NumeroMaxDeVoosPorAeronave)

    P_TurnaroundTime = df_escalares.TurnaroundTime[0]
    #print("P_TurnaroundTime = ",P_TurnaroundTime)

    P_AtrasoLimite1 = df_escalares.AtrasoLimite1[0]  
    #print("P_AtrasoLimite1 = ",P_AtrasoLimite1)

    P_AtrasoLimite2 = df_escalares.AtrasoLimite2[0]  
    #print("P_AtrasoLimite2 = ",P_AtrasoLimite2)

    P_InicioDoHorizonteDatetime = df_escalares.InicioDoHorizonte[0]
    #print("P_InicioDoHorizonteDatetime = ",P_InicioDoHorizonteDatetime)
    P_InicioDoHorizonte = Func_ConversaoDatetimeParaMinutosEscalares(P_InicioDoHorizonteDatetime)
    #print("P_InicioDoHorizonte = ",P_InicioDoHorizonte)

    P_FimDoHorizonteDatetime = df_escalares.FimDoHorizonte[0]
    #print("P_FimDoHorizonteDatetime = ",P_FimDoHorizonteDatetime)
    P_FimDoHorizonte = Func_ConversaoDatetimeParaMinutosEscalares(P_FimDoHorizonteDatetime)
    #print("P_FimDoHorizonte = ",P_FimDoHorizonte)

    P_PenCancelamentoComitiva = df_escalares.PenCancelamentoComitiva[0] 
    #print("P_PenCancelamentoComitiva = ",P_PenCancelamentoComitiva.value)

    P_PenCancelamento2Dias = df_escalares.PenCancelamento2Dias[0]
    #print("P_PenCancelamento2Dias = ",P_PenCancelamento2Dias.value)

    P_PenCancelamento1Dia = df_escalares.PenCancelamento1Dia[0]
    #print("P_PenCancelamento1Dia = ",P_PenCancelamento1Dia.value)

    P_PenCancelamentoTabela = df_escalares.PenCancelamentoTabela[0] 
    #print("P_PenCancelamentoTabela = ",P_PenCancelamentoTabela.value)

    P_PenUsoAeronaveSpot = df_escalares.PenUsoAeronaveSpot[0]
    #print("P_PenUsoAeronaveSpot = ",P_PenUsoAeronaveSpot.value)

    P_PenUsoAeronavePool = df_escalares.PenUsoAeronavePool[0]
    #print("P_PenUsoAeronavePool = ",P_PenUsoAeronavePool.value)

    P_PenUsoAeronaveComum = df_escalares.PenUsoAeronaveComum[0] 
    #print("P_PenUsoAeronaveComum = ",P_PenUsoAeronaveComum.value)

    P_PenTrocaAeronave = df_escalares.PenTrocaAeronave[0] 
    #print("P_PenTrocaAeronave = ",P_PenTrocaAeronave.value)

    P_PenAtrasoTipo2 = df_escalares.PenAtrasoTipo2[0]
    #print("P_PenAtrasoTipo2 = ",P_PenAtrasoTipo2.value)

    P_PenAtrasoTipo1 = df_escalares.PenAtrasoTipo1[0]
    #print("P_PenAtrasoTipo1 = ",P_PenAtrasoTipo1.value)

    P_PenAtrasoMinuto = df_escalares.PenAtrasoMinuto[0]
    #print("P_PenAtrasoMinuto = ",P_PenAtrasoMinuto.value)

    #2.2.2 Indexados
    #Convertendo para um dicionário, mas o índice deve ser igual ao nome da coluna
    #Horário Inicial
    #   Conversao de datetime para minutos
    P_HorarioInicialDatetime_dict = df_proginicial.set_index('Voo')['HorarioInicial'].to_dict()
    P_HorarioInicial_dict = Func_ConversaoDatetimeParaMinutosParametros(P_HorarioInicialDatetime_dict)
    #   Alteração do horário inicial para ser sempre 07:00
    for voo in S_VoosReais:
        if P_HorarioInicial_dict[voo] < 420:
            P_HorarioInicial_dict[voo] = 420

    #   Transformação de lista em parâmetro para modelo
    model.P_HorarioInicial = pyo.Param(S_VoosReais, initialize=P_HorarioInicial_dict)
    P_HorarioInicial = model.P_HorarioInicial
    #print("P_HorarioInicial = ",P_HorarioInicial.display())

    #UM Destino
    P_UMDestino_dict = df_proginicial.set_index('Voo')['UMDestino'].to_dict()
    model.P_UMDestino = pyo.Param(S_VoosReais, initialize=P_UMDestino_dict)
    P_UMDestino = model.P_UMDestino  
    #print("P_UMDestino = ",P_UMDestino.display())

    #Duração do Voo
    #   Conversao de datetime para minutos
    P_DuracaoDoVooDatetime_dict = df_proginicial.set_index('Voo')['DuracaoDoVoo'].to_dict()
    P_DuracaoDoVoo_dict = Func_ConversaoDatetimeParaMinutosParametros(P_DuracaoDoVooDatetime_dict)
    #   Transformação de lista em parâmetro para modelo
    model.P_DuracaoDoVoo = pyo.Param(S_VoosReais, initialize=P_DuracaoDoVoo_dict)
    P_DuracaoDoVoo = model.P_DuracaoDoVoo  
    #print("P_DuracaoDoVoo= ",P_DuracaoDoVoo.display())

    # Aeronave inicial
    P_AeronaveInicialString_dict = df_proginicial.set_index('Voo')['AeronaveInicial'].to_dict()
    # Convertendo para um parâmetro de 2 índices binário, em formato de dicionário
    P_AeronaveInicial = {}
    for i in S_VoosReais:
        for h in S_Aeronaves:
            if P_AeronaveInicialString_dict[i] == h:
                P_AeronaveInicial[i,h] = 1
            else:
                P_AeronaveInicial[i,h] = 0

    # Declarando o dicionário como parâmetro         
    model.P_AeronaveInicial = pyo.Param(S_VoosReais, S_Aeronaves, initialize=P_AeronaveInicial)
    P_AeronaveInicial = model.P_AeronaveInicial
    #print("P_AeronaveInicial = ",P_AeronaveInicial.display())

    # Tempo de Permanência
    #   Conversao de datetime para minutos
    P_TempoDePermanenciaDatetime_dict = df_proginicial.set_index('Voo')['TempoDePermanencia'].to_dict()
    P_TempoDePermanencia_dict = Func_ConversaoDatetimeParaMinutosParametros(P_TempoDePermanenciaDatetime_dict)
    #   Transformação de lista em parâmetro para modelo
    model.P_TempoDePermanencia = pyo.Param(S_VoosReais, initialize=P_TempoDePermanencia_dict)
    P_TempoDePermanencia = model.P_TempoDePermanencia 
    #print("P_TempoDePermanencia = ",P_TempoDePermanencia.display())

    # Tipo de Aeronave
    P_TipoDeAeronave_dict = df_aeronaves.set_index('Aeronave')['TipoDaAeronave'].to_dict()
    model.P_TipoDeAeronave = pyo.Param(S_Aeronaves, initialize=P_TipoDeAeronave_dict)
    P_TipoDeAeronave = model.P_TipoDeAeronave 
    #print("P_TipoDeAeronave = ",P_TipoDeAeronave.display())

    # Compatibilidade
    # Como esse parâmetro possui mais de um índice, é necessária uma manipulação prévia
    # Além disso, o dataframe com os dados deste parâmetro está num formato em que um dos indices (S_Voos) está na coluna e o outro (S_Aeronaves) está como cabeçalho
    # Para isso, os passos abaixo são seguidos:
    # Passo 1) usar a função .melt para alterar o formato do dataframe, transferindo o cabeçalho para uma nova coluna:
    df_compatibilidade_mod= df_compatibilidade.melt(id_vars=['Voo'], var_name='Aeronaves', value_name='Compatibilidade')
    # Passo 2) Create the dictionary in the (S_VoosReais, S_UM): Value format, que é o formato que a função pyo.Param aceita os dados de um parâmetro multiíndice
    P_Compatibilidade_dict = {(row['Voo'], row['Aeronaves']): row['Compatibilidade'] for _, row in df_compatibilidade_mod.iterrows()}
    # Passo 3) Por fim, realizar a atribuição ao parâmetro normalmente
    model.P_Compatibilidade = pyo.Param(S_VoosReais, S_Aeronaves, initialize=P_Compatibilidade_dict)
    P_Compatibilidade = model.P_Compatibilidade 
    # print("P_Compatibilidade = ",P_Compatibilidade.display())

    ##################################################################################################### Variáveis e FO
    # Variáveis do modelo
    # X(i,P,h)
    model.V_X3 = pyo.Var(S_VoosReais, S_Posicoes, S_Aeronaves, domain = pyo.Binary, bounds = [0,1])
    V_X3 = model.V_X3 
    #Y(i,h)
    #model.V_Y = pyo.Var(S_VoosReais, S_Aeronaves, domain = pyo.Binary, bounds = [0,1])
    #V_Y = model.V_Y 
    #Z(i,j)
    model.V_Z = pyo.Var(S_VoosReais, S_VoosReais, domain = pyo.Binary, bounds = [0,1])
    V_Z = model.V_Z
    #V(h)
    #model.V_V = pyo.Var(S_Aeronaves, domain = pyo.Binary, bounds = [0,1])
    #V_V= model.V_V
    #BI(i)
    model.V_BI = pyo.Var(S_VoosReais, domain = pyo.Binary, bounds = [0,1])
    V_BI = model.V_BI
    #BII(i)
    model.V_BII = pyo.Var(S_VoosReais, domain = pyo.Binary, bounds = [0,1])
    V_BII = model.V_BII
    #DT(i)
    model.V_DT = pyo.Var(S_VoosReais, domain = pyo.NonNegativeReals)
    V_DT = model.V_DT
    #AT(i)
    model.V_AT = pyo.Var(S_VoosReais, domain = pyo.NonNegativeReals)
    V_AT = model.V_AT
    #D(i)
    model.V_D = pyo.Var(S_VoosReais, domain = pyo.NonNegativeReals)
    V_D = model.V_D

    #Funcao Objetivo
    V_FO = (
            P_PenCancelamentoComitiva * (len(S_VoosComitiva) - sum(V_X3[vooC,p,h] for vooC in S_VoosComitiva for p in S_Posicoes for h in S_Aeronaves))
        +P_PenCancelamento2Dias * (len(S_VoosT2) - sum(V_X3[vooT2,p,h] for vooT2 in S_VoosT2 for p in S_Posicoes for h in S_Aeronaves))
        +P_PenCancelamento1Dia * (len(S_VoosT1) - sum(V_X3[vooT1,p,h] for vooT1 in S_VoosT1 for p in S_Posicoes for h in S_Aeronaves))
        +P_PenCancelamentoTabela * (len(S_VoosTabela) - sum(V_X3[vooTab,p,h] for vooTab in S_VoosTabela for p in S_Posicoes for h in S_Aeronaves))
        +P_PenUsoAeronaveSpot * sum(V_X3[voo,S_Posicoes.at(1),hSpot] for voo in S_VoosReais for hSpot in S_AeronavesSpot)
        +P_PenUsoAeronavePool * sum(V_X3[voo,S_Posicoes.at(1),hPool] for voo in S_VoosReais for hPool in S_AeronavesPool)
        +P_PenUsoAeronaveComum * sum(V_X3[voo,S_Posicoes.at(1),hComum] for voo in S_VoosReais for hComum in S_AeronavesComum)
        +P_PenAtrasoTipo2 * sum(V_BII[vooTabelaComitiva] for vooTabelaComitiva in S_VoosTabelaComitiva)
        +P_PenAtrasoTipo1 * sum(V_BI[vooTabelaComitiva] for vooTabelaComitiva in S_VoosTabelaComitiva)
        +P_PenAtrasoMinuto * sum(V_D[vooTodos] for vooTodos in S_VoosReais)
        +P_PenTrocaAeronave * sum(V_X3[vooTabelaComitiva,p,h] * (1 - P_AeronaveInicial[vooTabelaComitiva,h]) for vooTabelaComitiva in S_VoosTabelaComitiva for p in S_Posicoes for h in S_Aeronaves)
    )

    model.obj = pyo.Objective(expr = V_FO, sense = pyo.minimize)

    # Restrições ################################################################################################
    # 4.58 C_UmVooPorPosicaoAeronave
    model.C_UmVooPorPosicaoAeronave = pyo.Constraint(S_VoosReais,
                                                rule = lambda model, i: 
                                                    sum(V_X3[i,p,h] for p in S_Posicoes for h in S_Aeronaves)
                                                        <= 
                                                        1)
    # 4.59 C_ReciprocaUmVooPorPosicaoAeronave
    model.C_ReciprocaUmVooPorPosicaoAeronave = pyo.Constraint(S_Posicoes, S_Aeronaves,
                                                rule = lambda model, p,h: 
                                                    sum(V_X3[i,p,h] for i in S_VoosReais)
                                                        <= 
                                                        1)

    # 4.60 C_Precedencia
    model.C_Precedencia = pyo.Constraint(S_PosicoesExcetoUltima,S_Aeronaves,
                                                rule = lambda model, p, h: 
                                                    sum(V_X3[i,p,h] for i in S_VoosReais)
                                                        >=
                                                    #sum(V_X3[j,p_aux,h] for j in S_VoosReais if p_aux == p+1))
                                                    sum(V_X3[j,p+1,h] for j in S_VoosReais))

    # 4.61 C_OcupacaoComitiva
    model.C_OcupacaoComitiva = pyo.Constraint(S_VoosComitiva, S_VoosReais, S_PosicoesExcetoUltima, S_Aeronaves,
                                                rule = lambda model, i,j,p,h: 
                                                    sum(V_X3[j,p_aux,h] for p_aux in S_Posicoes if p_aux >= p+1)
                                                        <=
                                                        1 - V_X3[i,p,h])
    
     # 4.62 C_SequenciaCronologiCA
    model.C_SequenciaCronologica = pyo.Constraint(S_VoosReais, S_VoosReais, S_Aeronaves, S_PosicoesExcetoUltima,
                                                        rule = lambda model, i,j, h, p: 
                                                        V_DT[j]
                                                            >=
                                                        V_AT[i] 
                                                        + P_TurnaroundTime
                                                        - P_FimDoHorizonte * (2 - V_X3[i,p,h] - V_X3[j,p+1,h])
                                                        #if i != j else pyo.Constraint.Skip
                                                        )
    # 4.63 C_HorarioDeChegada
    model.C_HorarioDeChegada = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        V_AT[i]
                                                            ==
                                                        V_DT[i] + P_DuracaoDoVoo[i]*sum(V_X3[i,p,h] for p in S_Posicoes for h in S_Aeronaves))
    # 4.64 C_ChegadaDentroDaJanelaDeTempo - ok -tc5
    model.C_ChegadaDentroDaJanelaDeTempo = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        V_AT[i]
                                                            <=
                                                        P_FimDoHorizonte * sum(V_X3[i,p,h] for p in S_Posicoes for h in S_Aeronaves)) 
    # 4.65 C_PartidaAposOHorarioPrevisto
    model.C_PartidaAposOHorarioPrevisto = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        P_HorarioInicial[i] * sum(V_X3[i,p,h] for p in S_Posicoes for h in S_Aeronaves) 
                                                            <=
                                                        V_DT[i])
    # 4.66 C_AtivacaoDaPrecedencia1
    model.C_AtivacaoDaPrecedencia1 = pyo.Constraint(S_VoosReais, S_VoosReais,
                                                        rule = lambda model, i,j: 
                                                        V_DT[j]
                                                        -
                                                        V_DT[i]
                                                        <=
                                                        P_FimDoHorizonte*(V_Z[i,j] + 2 - sum(V_X3[i,p,h] for p in S_Posicoes for h in S_Aeronaves) - sum(V_X3[j,p,h] for p in S_Posicoes for h in S_Aeronaves))
                                                        if i != j else pyo.Constraint.Skip # garante que a restrição só será criada se i != j 
                                                    )

    # 4.67 C_ApenasUmaPrecedencia
    model.C_ApenasUmaPrecedencia = pyo.Constraint(S_VoosReais, S_VoosReais,
                                                        rule = lambda model, i,j: 
                                                        V_Z[i,j] + V_Z[j,i]
                                                        <=
                                                        1
                                                        if i != j else pyo.Constraint.Skip # garante que a restrição só será criada se i != j 
                                                    )
    # 4.68 C_AtivacaoDaPrecedencia2
    model.C_AtivacaoDaPrecedencia2 = pyo.Constraint(S_VoosReais, S_VoosReais,
                                                        rule = lambda model, i,j: 
                                                        V_Z[i,j] + V_Z[j,i]
                                                        >=
                                                        sum(V_X3[i,p,h] for p in S_Posicoes for h in S_Aeronaves) + sum(V_X3[j,p,h] for p in S_Posicoes for h in S_Aeronaves) - 1
                                                        if i != j else pyo.Constraint.Skip # garante que a restrição só será criada se i != j 
                                                    )
    # 4.69 C_DesigualdadeValida
    model.C_DesigualdadeValida = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        sum(V_Z[i,j] for j in S_VoosReais)
                                                        <=
                                                        len(S_VoosReais) * sum(V_X3[i,p,h] for p in S_Posicoes for h in S_Aeronaves)
                                                    )     
    # 4.70 C_TipoDeAtraso
    model.C_TipoDeAtraso = pyo.Constraint(S_VoosTabelaComitiva,
                                                        rule = lambda model, i0C: 
                                                        V_BI[i0C] + V_BII[i0C]
                                                        <=
                                                        1)
    # 4.71 C_ValorDoAtraso
    model.C_ValorDoAtraso = pyo.Constraint(S_VoosTabelaComitiva,
                                                        rule = lambda model, i0C: 
                                                        V_D[i0C]
                                                        <=
                                                        P_AtrasoLimite1 * V_BI[i0C] + P_AtrasoLimite2 * V_BII[i0C]
                                                        ) 
    # 4.72 C_DeterminaValorDePartida
    model.C_DeterminaValorDePartida = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        V_DT[i]
                                                            <=
                                                        P_HorarioInicial[i] * sum(V_X3[i,p,h] for p in S_Posicoes for h in S_Aeronaves) + V_D[i])
    # 4.73 C_TempoDePermanencia - ok - oc2
    model.C_TempoDePermanencia = pyo.Constraint(S_VoosReais,S_VoosReais,S_UM,
                                                        rule = lambda model, i,j,um: 
                                                        V_DT[j] - V_DT[i]
                                                        >=
                                                        P_TempoDePermanencia[i] * V_Z[i,j] - P_FimDoHorizonte * (1 - V_Z[i,j])
                                                        if ((i != j) and (P_UMDestino[i] == P_UMDestino[j])) else pyo.Constraint.Skip
                                                    )
    # 4.74 C_TempoDeSeguranca - ok - oc1
    model.C_TempoDeSeguranca = pyo.Constraint(S_VoosReais,S_VoosReais,
                                                        rule = lambda model, i,j: 
                                                        V_DT[j] - V_DT[i]
                                                        >=
                                                        P_TempoDeSeguranca * V_Z[i,j] - P_FimDoHorizonte * (1 - V_Z[i,j])
                                                        if i != j else pyo.Constraint.Skip
                                                    )

    # Voo i incompatível com aeronave h para V_X3
    for i in S_VoosReais:
        for h in S_Aeronaves:
            for p in S_Posicoes:
                if P_Compatibilidade[i,h] == 0:
                    V_X3[i,p,h].fix(0)


    # Respeitar precedência de voos

    for i0 in S_VoosTabela:
        for itransf in S_VoosTransferencia:
    #       for um in S_UM:
                if P_UMDestino[i0] == P_UMDestino[itransf] and i0 != itransf :
                    V_Z[i0,itransf].fix(0)      

    # Informações do Modelo
    # Count all variables in the model
    P_total_variables[instancia] = len(list(model.component_data_objects(Var)))

    # Count integer variables
    P_integer_variables[instancia] = len([v for v in model.component_data_objects(Var) if v.domain == Integers])

    # Count binary variables
    P_binary_variables[instancia] = len([v for v in model.component_data_objects(Var) if v.domain == Binary])

    # Count all constraints in the model
    P_total_constraints[instancia] = len(list(model.component_data_objects(Constraint)))

    # Print the results
    print(f"Total number of variables: {P_total_variables[instancia]}")
    print(f"Total number of integer variables: {P_integer_variables[instancia]}")
    print(f"Total number of binary variables: {P_binary_variables[instancia]}")
    print(f"Total number of constraints: {P_total_constraints[instancia]}")

    #opt = SolverFactory(solver_selecionado, executable='C:\\Users\\mateu\\Desktop\\Estudos\\Solvers\\SCIP\\scip-9.2.0\\SCIPOptSuite-9.2.0-win64.exe')
    opt = SolverFactory(solver_selecionado)

    # Verificando se o solver selecionado existe
    #print(opt.available())

    # Nome da rodada
    Ps_NomeDaRodada[instancia] = Ps_NomeDaInstancia + "_" + solver_selecionado + "_" + f"{P_HoraDaRodadaTimeStamp}"

    # Definindo as opções do solver
    # Set solver options for GLPK
    if solver_selecionado == 'glpk':
        opt.options['tmlim'] = 3600  # Set time limit (seconds)
        opt.options['mipgap'] = 0.00  # Set optimality gap (percentage)
        #opt.options['threads'] = 4  # Set the number of threads
        #solver.options['log'] = f"{Ps_NomeDaRodada}.txt"  # Optional: Log file (GLPK doesn't directly support this in Pyomo)

    # CBC
    if solver_selecionado == 'cbc':
        opt.options['seconds'] = 3600  # Set a time limit (optional)
        opt.options['ratio'] = 0.00  # Set optimality gap (optional)
        opt.options['threads'] = 4  # set number of threads (optional)
        opt.options['log'] = 1  # Enable verbose logging
        opt.options['allowableGap'] = 0 # Ensure the absolute gap is 0
        #opt.options['logFile'] = f"{Ps_NomeDaRodada}.txt" # escreve o log da rodada

    #Gurobi
    if solver_selecionado == 'gurobi':
        opt.options['TimeLimit'] = 3600  # Set a time limit (optional)
        opt.options['MIPGap'] = 0.00  # Set optimality gap (optional)
        opt.options['Threads'] = 4  # set number of threads (optional)
        #opt.options['LogFile'] = f"{Ps_NomeDaRodada}.txt" # escreve o log da rodada

    # CPLEX
    if solver_selecionado == 'cplex':
        opt.options['TimeLimit'] = 3600  # Set a time limit (optional)
        opt.options['mip_tolerances_mipgap'] = 0.00  # Set optimality gap (optional)
        opt.options['threads'] = 4 # set number of threads (optional)
        #opt.options['LogFile'] = f"{Ps_NomeDaRodada}.txt" # escreve o log da rodada

    # HiGHS
    if (solver_selecionado == 'appsi_highs') or (solver_selecionado == 'highs'):
        opt.options['time_limit'] = 3600  # Set a time limit (optional)
        opt.options['mip_rel_gap'] = 0.00  # Set optimality gap (optional)
        opt.options['threads'] = 4 # set number of threads (optional)
        #opt.options['log_file'] = f"{Ps_NomeDaRodada}.txt" # escreve o log da rodada

    # MOSEK
    if solver_selecionado == 'mosek':
        # Set logging file (this is for MOSEK output)
        #opt.options['logfile'] = f"{Ps_NomeDaRodada}.txt"  # Log file path
        # Set time limit (in seconds)
        opt.options['dparam.optimizer_max_time'] = 3600  # 1 hour limit
        # Set the optimality gap (optional)
        opt.options['dparam.mio_rel_gap_const'] = 0.00001  # 0% gap for exact solution
        # Set number of threads (optional)
        opt.options['iparam.num_threads'] = 4  # Set to 4 threads
    
    # SCIP
    if solver_selecionado == 'scip':       
        # Set SCIP options
        opt.options['limits/time'] = 3600         # Set time limit to 3600 seconds
        opt.options['limits/gap'] = 0.00         # Set optimality gap
        opt.options['parallel/maxnthreads'] = 4  # Use 4 threads

    # Solving the model
    # Relaxação linear
    if relaxacao_linear == 1:
        xfrm = TransformationFactory('core.relax_integer_vars')
        relaxed_model = xfrm.apply_to(model)
    
    start_time[instancia] = datetime.now()
    results = opt.solve(model, tee=True)
    #results = opt.solve(model)   
    end_time[instancia] = datetime.now()
    # Resultados
    print("Resultados da rodada: ", Ps_NomeDaRodada[instancia], f"Para o solver {solver_selecionado}")
    # Retrieve the objective value of the incumbent solution (best integer solution found)
    P_SOL_incumbent_solution[instancia] = model.obj()

    # Retrieve solver results
    P_SOL_solver_status[instancia] = results.solver.status
    #P_SOL_termination_condition[instancia] = results.solver.termination_condition
    P_SOL_solver_time[instancia] = results.solver.time if hasattr(results.solver, 'time') else None # Time spent solving the model    
    
    # Retrieve the best bound (only for MIP problems)
    P_SOL_best_bound[instancia] = results.solver.best_bound if hasattr(results.solver, 'best_bound') else None

    # Retrieve the optimality gap (only for MIP problems)
    P_SOL_optimality_gap[instancia] = results.solver.optimality_gap if hasattr(results.solver, 'optimality_gap') else None

    #P_SOL_optimality_gap = results.solver.get('mipgap', None)

    # Output the results
    print(f"Incumbent Solution: {P_SOL_incumbent_solution[instancia]}")
    #print(f"Best Bound: {P_SOL_best_bound}")
    #print(f"Optimality Gap: {P_SOL_optimality_gap}")
    print(f"Solver Status: {P_SOL_solver_status[instancia]}")
    #print(f"Termination Condition: {P_SOL_termination_condition}")
    print(f"Solver Time: {P_SOL_solver_time[instancia]} seconds")     

    elapsed_time[instancia] = end_time[instancia] - start_time[instancia]                                      

[1, 3, 4, 6, 7, 8, 9, 10]
Ps_NomeDaInstancia =  i11_Simulado
S_Posicoes =  [1, 2, 3, 4, 5]
S_PosicoesExcetoUltima =  [1, 2, 3, 4]
P_DuracaoDoVoo : Size=11, Index=S_VoosReais, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :   104
      2 :    93
      3 :    91
      4 :   103
      5 :    79
      6 :    87
      7 :    85
      8 :    87
      9 :    73
     10 :   100
     11 :    79
P_DuracaoDoVoo=  None
Ps_NomeDaInstancia =  i11_Simulado
(type=<class 'pyomo.core.base.param.IndexedParam'>) on block unknown with a
new Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 

<string>:14: SyntaxWarning: invalid escape sequence '\{'
<>:14: SyntaxWarning: invalid escape sequence '\{'
<>:14: SyntaxWarning: invalid escape sequence '\M'
<string>:14: SyntaxWarning: invalid escape sequence '\{'
<>:14: SyntaxWarning: invalid escape sequence '\{'
<>:14: SyntaxWarning: invalid escape sequence '\M'
C:\Users\mateu\AppData\Local\Temp\ipykernel_27228\1219184466.py:14: SyntaxWarning: invalid escape sequence '\{'
  Ps_CaminhoDaPlanilha = f"G:\Meu Drive\Planilhas para Estudos\{instancia}.xlsx"
C:\Users\mateu\AppData\Local\Temp\ipykernel_27228\1219184466.py:14: SyntaxWarning: invalid escape sequence '\M'
  Ps_CaminhoDaPlanilha = f"G:\Meu Drive\Planilhas para Estudos\{instancia}.xlsx"


Total number of variables: 341
Total number of integer variables: 0
Total number of binary variables: 308
Total number of constraints: 2071
Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - C:\Users\mateu\Desktop\Estudos\Solvers\CBC\Cbc-releases.2.10.12-w64-msvc17-md\bin\cbc.exe -seconds 3600 -ratio 0.0 -threads 4 -printingOptions all -import C:\Users\mateu\AppData\Local\Temp\tmpumtt8gw9.pyomo.lp -stat=1 -solve -solu C:\Users\mateu\AppData\Local\Temp\tmpumtt8gw9.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 3600
ratioGap was changed from 0 to 0
No match for threads - ? for list of commands
No match for 4 - ? for list of commands
Option for printingOptions changed from normal to all
Presolve 1795 (-276) rows, 272 (-12) columns and 10271 (-1755) elements
Statistics for presolved model
Original problem has 250 integers (250 of which binary)
Presolved problem has 239 integers (239 of which binary)
==== 130 zero objective 16 diff

# Exportação

In [24]:
elapsed_time_minutes = Func_ConversaoDatetimeParaMinutosParametros(elapsed_time)
print(elapsed_time_minutes)

{'i11': datetime.timedelta(seconds=495, microseconds=333732)}


In [26]:

# Exportação
df_export = pd.DataFrame({"P_SOL_NomeDaRodada": Ps_NomeDaRodada,
                            "P_SOL_incumbent_solution": P_SOL_incumbent_solution, 
                          "P_SOL_solver_status": P_SOL_solver_status, 
                          "P_SOL_solver_time": P_SOL_solver_time,
                          "elapsed_time_minutes" : elapsed_time_minutes,
                          "P_SOL_best_bound": P_SOL_best_bound,
                          "P_SOL_optimality_gap": P_SOL_optimality_gap,
                          "P_total_variables" : P_total_variables,
                          "P_integer_variables" : P_integer_variables,
                          "P_binary_variables": P_binary_variables,
                          "P_total_constraints" : P_total_constraints
                          })
print(df_export)

for i in S_VoosReais:
    for p in S_Posicoes:
        for h in S_Aeronaves:
            if V_X3[i,p,h].value > 0:
                print(f"V_X3({i},{p},{h}) = ", V_X3[i,p,h].value)

for i in S_VoosReais:
  if V_DT[i].value > 0:
      print(f"V_DT({i}) = ", V_DT[i].value)

# Penalidades FO
P_SOL_CancelamentoComitiva = P_PenCancelamentoComitiva * (len(S_VoosComitiva) - sum(V_X3[vooC,p,h].value for vooC in S_VoosComitiva for p in S_Posicoes for h in S_Aeronaves))
P_SOL_PenCancelamento2Dias = P_PenCancelamento2Dias * (len(S_VoosT2) - sum(V_X3[vooT2,p,h].value for vooT2 in S_VoosT2 for p in S_Posicoes for h in S_Aeronaves))
P_SOL_PenCancelamento1Dia =        P_PenCancelamento1Dia * (len(S_VoosT1) - sum(V_X3[vooT1,p,h].value for vooT1 in S_VoosT1 for p in S_Posicoes for h in S_Aeronaves))
P_SOL_PenCancelamentoTabela =        P_PenCancelamentoTabela * (len(S_VoosTabela) - sum(V_X3[vooTab,p,h].value for vooTab in S_VoosTabela for p in S_Posicoes for h in S_Aeronaves))
P_SOL_PenUsoAeronaveSpot    =  P_PenUsoAeronaveSpot * sum(V_X3[voo,S_Posicoes.at(1),hSpot].value for voo in S_VoosReais for hSpot in S_AeronavesSpot)
P_SOL_PenUsoAeronavePool    =  P_PenUsoAeronavePool * sum(V_X3[voo,S_Posicoes.at(1),hPool].value for voo in S_VoosReais for hPool in S_AeronavesPool)
P_SOL_PenUsoAeronaveComum   = P_PenUsoAeronaveComum * sum(V_X3[voo,S_Posicoes.at(1),hComum].value for voo in S_VoosReais for hComum in S_AeronavesComum)
P_SOL_PenAtrasoTipo2 =  P_PenAtrasoTipo2 * sum(V_BII[vooTabelaComitiva].value for vooTabelaComitiva in S_VoosTabelaComitiva)
P_SOL_PenAtrasoTipo1 =  P_PenAtrasoTipo1 * sum(V_BI[vooTabelaComitiva].value for vooTabelaComitiva in S_VoosTabelaComitiva)
P_SOL_PenAtrasoMinuto = P_PenAtrasoMinuto * sum(V_D[vooTodos].value for vooTodos in S_VoosReais)
P_SOL_PenTrocaAeronave = P_PenTrocaAeronave * sum(V_X3[vooTabelaComitiva,p,h].value * (1 - P_AeronaveInicial[vooTabelaComitiva,h]) for vooTabelaComitiva in S_VoosTabelaComitiva for p in S_Posicoes for h in S_Aeronaves)

P_NumeroDeVoos2 = len(S_VoosTabela)
P_SomatorioDeVoosAlocados = sum(V_X3[vooTab,p,h].value for vooTab in S_VoosTabela for p in S_Posicoes for h in S_Aeronaves)

print('P_NumeroDeVoos2 = ', P_NumeroDeVoos2)
print('P_SomatorioDeVoosAlocados = ', P_SomatorioDeVoosAlocados)
print('P_SOL_CancelamentoComitiva = ', P_SOL_CancelamentoComitiva)
print('P_SOL_PenCancelamento2Dias = ', P_SOL_PenCancelamento2Dias)
print('P_SOL_PenCancelamento1Dia = ', P_SOL_PenCancelamento1Dia)
print('P_SOL_PenCancelamentoTabela = ', P_SOL_PenCancelamentoTabela)
print('P_SOL_PenUsoAeronaveSpot = ', P_SOL_PenUsoAeronaveSpot)
print('P_SOL_PenUsoAeronavePool = ', P_SOL_PenUsoAeronavePool)
print('P_SOL_PenUsoAeronaveComum = ', P_SOL_PenUsoAeronaveComum)
print('P_SOL_PenAtrasoTipo2 = ', P_SOL_PenAtrasoTipo2)
print('P_SOL_PenAtrasoTipo1 = ', P_SOL_PenAtrasoTipo1)
print('P_SOL_PenAtrasoMinuto = ', P_SOL_PenAtrasoMinuto)
print('P_SOL_PenTrocaAeronave = ', P_SOL_PenTrocaAeronave)
#df_export.to_excel(f"DataFram_{Ps_NomeDaRodada}.xlsx", index = True)
#df_export.to_excel(f"G:\Meu Drive\Planilhas para Estudos\ResultadosPyomoM3\MIP_{solver_selecionado}_{P_HoraDaRodadaTimeStamp}.xlsx", index = True)7#

                  P_SOL_NomeDaRodada  P_SOL_incumbent_solution  \
i11  i11_Simulado_cbc_20250128180605                   147.134   

    P_SOL_solver_status  P_SOL_solver_time   elapsed_time_minutes  \
i11                  ok         495.238927 0 days 00:08:15.333732   

    P_SOL_best_bound P_SOL_optimality_gap  P_total_variables  \
i11             None                 None                341   

     P_integer_variables  P_binary_variables  P_total_constraints  
i11                    0                 308                 2071  
V_X3(1,1,PR-SHL) =  1.0
V_X3(2,2,PR-LCR) =  1.0
V_X3(4,3,PR-LDG) =  1.0
V_X3(5,1,PR-LCR) =  1.0
V_X3(6,1,PR-LDG) =  1.0
V_X3(7,2,PR-SHL) =  1.0
V_X3(8,4,PR-LCR) =  1.0
V_X3(9,2,PR-LDG) =  1.0
V_X3(10,3,PR-SHL) =  1.0
V_X3(11,3,PR-LCR) =  1.0
V_DT(1) =  420.0
V_DT(2) =  549.0
V_DT(4) =  900.0
V_DT(5) =  425.0
V_DT(6) =  530.0
V_DT(7) =  750.0
V_DT(8) =  870.0
V_DT(9) =  780.0
V_DT(10) =  930.0
V_DT(11) =  720.0
P_NumeroDeVoos2 =  8
P_SomatorioDeVoosAlocados = 

In [29]:
print(list(S_VoosTabela))
model.write('cbc.lp')

[1, 3, 4, 6, 7, 8, 9, 10]


('cbc.lp', 1750557934608)